# Analysis of timelapse data from Raspiscope

## Preliminaries
Before processing the images we need to set up the tools and load the data. We need to import several packages, so before running this notebook you should create an environment (conda or virtualenv) with matplotlib, numpy, and scikit-image, for example:

`conda create env -n raspiscope matplotlib numpy scikit-image`

and then activate it, e.g.:

`source activate raspiscope`

or select the conda env here in Jupyter.

### Modules
First import the standard tools, numpy and matplotlib. These are very well documented packages, more info can be found here:

http://www.matplotlib.org

http://www.numpy.org

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

We will use scikit-image to analyse and process images. There are many examples and tutorials here:

http://www.scitkit-image.org

In [ ]:
import skimage 
from skimage import io, filters

### Loading images
First we specify a text string with the location and name of image files in the timelapse. We put a %d to represent the frame number, %04d means put the number with 4 digits. Using the % operator on the string we can then form any file name in the series:

In [ ]:
#fname = "/Users/timrudge/RaspiScope/Timelapse060117/image_%04d.jpg"
#fname = "/Users/timrudge/RaspiScope/timelapse/timlap_3_%04d.jpg"
fname =  "/Users/timrudge/RaspiScope/Lab_19_1_17/image_%04d.jpg"
fname%(0)

Lets see what frame 500 looks like. We use PyPlot to read the file into a numpy array. This is a 3-dimensional array (a grid of values) with dimensions x,y,c, with (x,y) position and c the color channel (red, green, blue). 

<img src="files/imagearray.png" width=256 height=256>

We can use PyPlot again to display the array, which interprets the array as a color image correctly.

In [ ]:
im = plt.imread(fname%500)
plt.imshow(im)

### Loading the time series
There are 3 channels in each image. For now we will just look at the green channel (c=1). We will load the green channel of all the images into a 3-dimensional array, with dimensions x, y, time. 

<img src="files/timeseriesarray.png" width=256 height=256>

First get the size of the image from the numpy array shape, the 3rd dimension size is 3 for R,G,B. We dont need it so use the "_" variable.

In [ ]:
im.shape
w,h,_ = im.shape

Now we create a numpy array to hold the data and loop over the files, taking only channel 1 (green):

In [ ]:
nt = 552
ims = np.zeros((w,h,nt))
for i in range(nt):
    im = plt.imread(fname%i)
    ims[:,:,i] = im[:,:,1]

Now we have a stack (x,y,t):

In [ ]:
ims.shape

## Analysis of time-lapse images
Now we have all the data loaded into a numpy array. To speed things up, take a subregion of the image to analyse. We do this by '[slicing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)' the array:

In [ ]:
imssub = ims[200:600,200:600,:]
plt.imshow(imssub[:,:,-1])
plt.colorbar()

### Mean dynamics over image domain
We can get some information by looking just at the variation in total image intensity in the time-series. Do this by taking the sum of the (x,y,t) stack over the first two dimensions (0,1):

In [ ]:
sumims = imssub.sum(axis=(0,1))
plt.plot(sumims)

### Identifying colonies
The colonies vary in brightness over the experiment but remain stationary. So we can take the sum of intensity of each pixel over time, and use it to locate colonies reliably:

In [ ]:
ims_sum = imssub.sum(axis=2)
plt.imshow(ims_sum)
plt.colorbar()

With any image its essential to remove noise before analysis, here with a [Gaussian filter](http://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.gaussian). Normalisation to (0,1) is also convenient:

In [ ]:
from skimage.filters import gaussian
simsub = gaussian(ims_sum, 2)
nsimsub = (simsub-simsub.min())/(simsub.max()-simsub.min())

In [ ]:
plt.imshow(nsimsub)
plt.colorbar()

#### Blob detection
Scikit-image provides several functions to detect Gaussian-like features ([blobs](http://scikit-image.org/docs/dev/auto_examples/plot_blob.html)). We apply one of these ([blob_log](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log)) to the smoothed sum of intensity, and it returns a list of position (x,y) and width (standard deviation):

In [ ]:
import skimage.feature as skfeat
A = skfeat.blob_log(nsimsub, min_sigma=1.0, max_sigma=10.0, num_sigma=100, threshold=0.25, overlap=0.8)

In [ ]:
A.shape
A[1:4,:]

The array A now contains the location of probable colonies. We can check by showing the image and plotting markers on top at each location contained in A:

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(nsimsub, cmap='gray')
plt.hold(True)
for i in range(len(A)):
    plt.plot(A[i,1],A[i,0],'r+')

### Time dynamics of colonies
Now we have estimated colony location (and size) we can analyse the dynamics of fluorescence and growth. First to distinguish cell fluorescence from media and background light, we select an empty region of the plate (by eye) and take the mean value in each frame as the background:

In [ ]:
bg = imssub[300:400,0:100,:].mean(axis=(0,1))
plt.plot(bg)

To restrict image analysis for each colony, we can extract a region at each location, with size given by the Gaussian width (x2 = 2 standard deviations). Here we put the image series into a dict indexed by colony. We also subtract the background estimate from each time series.

In [ ]:
rois = {}
for i in range(len(A)):
    x = A[i,0]
    y = A[i,1]
    r = 2*A[i,2]
    rois[i] = imssub[x-r:x+r,y-r:y+r,:]-bg

Now we can plot the total fluoresence in the region of each colony:

In [ ]:
for i in range(len(A)):
    plt.plot(rois[i].sum(axis=(0,1)))
    plt.hold(True)

#### Colony radius growth
We have the location and some kind of size estimate based on the total fluorescence of each colony over the time-lapse. We would like to know the size of the colony in each frame; from this we can compute the growth rate.

One way to do this is the find Gaussian blobs in each of the image regions of the colony, for each frame of the time-lapse. Then assume that the first (only?) blob detected is the colony we are interested in. 

This is very slow, there is a simpler way since we can assume a single Gaussian in each image region. Anyway for now it works ok.

The following loop finds blobs and stores the width in each frame. The result is a dict containing the size at each time point for every colony.

In [ ]:
R = {}
for k in range(len(A)):
    R[k] = np.zeros((nt,))
    for i in range(nt):
        troi = rois[k][:,:,i].astype(np.float32)
        ntroi = (troi-troi.min())/(troi.max()-troi.min())
        AA = skfeat.blob_log(ntroi, min_sigma=1.0, max_sigma=10.0, num_sigma=200, threshold=0.25, overlap=0.8)
        if len(AA)>0:
            R[k][i] = AA[0,2]

Since the colony is roughly circular and flat, an estimate of volume is $R^2$. Lets plot this for some colonies. Plotting the log is useful because the slope is the relative growth rate. We can see the colonies slow their growth during the experiment.

In [ ]:
idx = [0,1,2,3,4,9]
for i in idx:
    r = R[i][0:-1:10]
    plt.plot(np.log(r*r))
    plt.hold(True)

#### Check the radius estimate
To see how well the colony size is estimated, we can compare to the profile of a colony in a slice of its image region. The radius (red line) should give the border of the colony.

In [ ]:
idx = 6
plt.figure(figsize=(16,12))
w,h,d = rois[idx].shape
plt.imshow(rois[idx][w/2+1,:,0:-1:10], interpolation='none', cmap='gray')
plt.colorbar()
plt.hold(True)
plt.plot(-R[idx][0:-1:10]*2+h/2+1,'r')
plt.plot(R[idx][0:-1:10]*2+h/2+1,'r')

### Dynamics plotting
A simple way to view dynamics is a movie, plotting each frame successively. To do this we have to turn off inline plotting so that a new window will open.

In [ ]:
%matplotlib auto
mx = np.max(rois[9])
for i in range(300,400):
    plt.imshow(rois[9][:,:,i], interpolation='none', vmin=0, vmax=mx)
    plt.title('%f'%R[9][i])
    plt.pause(0.1)
%matplotlib inline

### Parameter estimation from data
We should be able to follow the same process as in the ratiometric paper using platereader data. Here testing...

In [ ]:
for i in range(len(A)):
    plt.plot(R[i]*R[i], rois[i].sum(axis=(0,1)), '.')
    plt.hold(True)

In [ ]:
for i in range(len(A)):
    roi = rois[i]
    plt.plot(np.log(roi.mean(axis=(0,1))))
    plt.hold(True)
plt.legend()

In [ ]:
plt.plot(ims[500,200,:])

In [ ]:
plt.figure(figsize=(16,12))
plt.imshow(ims[A[10,0],400:450,:])

In [ ]:
s = np.diff(ims[A[10,0],:,:], axis=1)
s.shape

In [ ]:
c=np.cov(s)
x = A[10,0]
y = A[10,1]
plt.plot(c[y,y-20:y+20])
c[y,y]